In [1]:
# activate R magic
%load_ext rpy2.ipython

/usr/local/lib/python3.6/dist-packages/rpy2/robjects/pandas2ri.py:14: FutureWarning: pandas.core.index is deprecated and will be removed in a future version.  The public classes are available in the top-level namespace.
  from pandas.core.index import Index as PandasIndex
/usr/local/lib/python3.6/dist-packages/rpy2/robjects/pandas2ri.py:34: UserWarning: pandas >= 1.0 is not supported.
  warnings.warn('pandas >= 1.0 is not supported.')


Напишите функцию, all_to_factor, которая преобразует dataframe, переводя все его переменные в фактор.

In [0]:
%%R
library("dplyr")

In [0]:
%%R
all_to_factor <- function(x){
    x <- mutate_each(x, funs(as.factor))
    return(x)
}

В этом задании от вас потребуется написать функцию для предобработки данных log_transform. В статистике часто трансформируют исходные переменные. Например, используют значение натурального логарифма исходной переменной.

Ваша задача написать функцию, которая получает на вход dataframe  с произвольным числом переменных разных типов. На первом этапе функция должна выполнить предобработку числовых переменных. Т.к. значение логарифма мы можем рассчитать только для положительных чисел. Для этого сделаем центрирование всех переменных (Rescaling), только еще добавим единичку, чтобы у нас не осталось нулей:


После того как мы масштабировали каждую переменную, осталось рассчитать значение натурального логарифма каждого наблюдения (функция log) и вернуть новый dataframe. 
```
> test_data
    V1   V2   V3 V4
1  1.5 -0.9 -2.8  A
2 -0.1 -0.3 -3.1  B
3  2.5 -2.4 -1.8  B
4 -0.3  0.0  2.1  B
5 -0.8  0.4  1.9  B

> log_transform(test_data)

         V1        V2         V3 V4
1 0.5288441 0.4289956 0.05608947  A
2 0.1923719 0.5596158 0.00000000  B
3 0.6931472 0.0000000 0.22314355  B
4 0.1410786 0.6190392 0.69314718  B
5 0.0000000 0.6931472 0.67372909  B
```

In [0]:
%%R
log_transform <- function(df){
    num_var <- names(df[sapply(df, is.numeric)])
    df <- mutate_each(df, funs(log((. - min(.)) / (max(.) - min(.)) + 1)), num_var)
    return(df)
}


In [6]:
%%R 
df <- data_frame(V1 = c(1.5, -0.1, 2.5, -0.3, -0.8),
                        V2 = c(-0.9, -0.3, -2.4, 0, 0.4),
                        V3 = c(-2.8, -3.1, -1.8, 2.1, 1.9),
                        V4 = c("A", rep("B", 4)))
num_var <- names(df[sapply(df, is.numeric)])
num_var

[1] "V1" "V2" "V3"


В ситуации, когда у нас есть несколько факторов все наши наблюдения разбиваются на столько групп, сколько возможно комбинации уровней факторов. Например, если у нас есть данные, где количественная переменная - размер зарплаты, а два фактора это пол (мужчины и женщины) и национальность (англичане и французы), то два эти фактора разбивают наблюдения на четыре группы:  мужчины французы, женщины француженки, мужчины англичане и женщины англичанки.  В общем число всех возможных групп равняется произведению числа градацией факторов.


И так, ваша задача будет написать функцию descriptive_stats, которая рассчитывает основные описательные статистики в каждой группе наблюдений для описанного выше примера. Функция получает на вход dataframe с тремя переменными salary - значение заработной платы, gender - фактор с двумя градациями (male, female), country - фактор с двумя градациями (England, France).

Функция должна возвращать dataframe с описательными статистиками и количеством NA, рассчитанными в каждой группе: количеств наблюдений,﻿ среднее значение, стандартное отклонение, медиана, первый квартиль, третий квартиль, число пропущенных значений.

Воспользуйтесь функцией quantile() для расчета квартилей в данных.

Пример работы функции и формата dataframe c описательными статистиками.
```
> test_data <- read.csv("https://stepic.org/media/attachments/course/724/salary.csv")
> descriptive_stats(test_data)
  gender country     n     mean       sd   median first_quartile third_quartile na_values
  <fctr>  <fctr> <int>    <dbl>    <dbl>    <dbl>          <dbl>          <dbl>     <int>
1 Female England   128 60.26013 4.539153 60.13800       57.74711       63.24743        15
2 Female  France   122 60.52732 4.682239 61.07025       56.70696       64.34622         8
3   Male England   122 59.30351 4.601040 58.88415       56.72627       62.26308         6
4   Male  France   128 60.05907 5.000158 60.53287       56.20641       62.99529        14
```

In [0]:
%%R
descriptive_stats <- function (dataset){
    dataset %>% 
      group_by(gender, country) %>% 
        summarise(n = n(),
            mean = mean(salary, na.rm = T),
            sd = sd(salary, na.rm = T),
            median = median(salary, na.rm = T),
            first_quartile = quantile(salary, 0.25, na.rm = T),
            third_quartile = quantile(salary, 0.75, na.rm = T),
            na_values = sum(is.na(salary)))
}

Напишите функцию, to_factors, которая получает на вход dataframe  с произвольным числом количественных переменных и вектор с номерами колонок, которые нужно перевести в фактор.

Для перевода числовых колонок в фактор будем использовать следующий принцип, если наблюдение больше среднего всей переменной то 1, иначе 0.

```
> to_factors(mtcars[1:4], factors = c(1, 3))
                    mpg cyl disp  hp
Mazda RX4             1   6    0 110
Mazda RX4 Wag         1   6    0 110
Datsun 710            1   4    0  93
Hornet 4 Drive        1   6    1 110
Hornet Sportabout     0   8    1 175
Valiant               0   6    0 105
Duster 360            0   8    1 245
Merc 240D             1   4    0  62
Merc 230              1   4    0  95
Merc 280              0   6    0 123
Merc 280C             0   6    0 123
Merc 450SE            0   8    1 180
Merc 450SL            0   8    1 180
Merc 450SLC           0   8    1 180
Cadillac Fleetwood    0   8    1 205
Lincoln Continental   0   8    1 215
Chrysler Imperial     0   8    1 230
Fiat 128              1   4    0  66
Honda Civic           1   4    0  52
Toyota Corolla        1   4    0  65
Toyota Corona         1   4    0  97
Dodge Challenger      0   8    1 150
AMC Javelin           0   8    1 150
Camaro Z28            0   8    1 245
Pontiac Firebird      0   8    1 175
Fiat X1-9             1   4    0  66
Porsche 914-2         1   4    0  91
Lotus Europa          1   4    0 113
Ford Pantera L        0   8    1 264
Ferrari Dino          0   6    0 175
Maserati Bora         0   8    1 335
Volvo 142E            1   4    0 109
```

Убедитесь также, что ваша функция корректно работает, если получает на вход номер только одной колонки для преобразования.

In [11]:
%%R 
str(mtcars)
to_factors <- function(test_data, factors){
  test_data %>% mutate_at(factors, funs(as.factor(ifelse(. > mean(.), 1, 0 ))))
}
to_factors(mtcars[1:4], factors = c(1, 3))

'data.frame':	32 obs. of  11 variables:
 $ mpg : num  21 21 22.8 21.4 18.7 18.1 14.3 24.4 22.8 19.2 ...
 $ cyl : num  6 6 4 6 8 6 8 4 4 6 ...
 $ disp: num  160 160 108 258 360 ...
 $ hp  : num  110 110 93 110 175 105 245 62 95 123 ...
 $ drat: num  3.9 3.9 3.85 3.08 3.15 2.76 3.21 3.69 3.92 3.92 ...
 $ wt  : num  2.62 2.88 2.32 3.21 3.44 ...
 $ qsec: num  16.5 17 18.6 19.4 17 ...
 $ vs  : num  0 0 1 1 0 1 0 1 1 1 ...
 $ am  : num  1 1 1 0 0 0 0 0 0 0 ...
 $ gear: num  4 4 4 3 3 3 3 4 4 4 ...
 $ carb: num  4 4 1 1 2 1 4 2 2 4 ...
   mpg cyl disp  hp
1    1   6    0 110
2    1   6    0 110
3    1   4    0  93
4    1   6    1 110
5    0   8    1 175
6    0   6    0 105
7    0   8    1 245
8    1   4    0  62
9    1   4    0  95
10   0   6    0 123
11   0   6    0 123
12   0   8    1 180
13   0   8    1 180
14   0   8    1 180
15   0   8    1 205
16   0   8    1 215
17   0   8    1 230
18   1   4    0  66
19   1   4    0  52
20   1   4    0  65
21   1   4    0  97
22   0   8    1 150
23   

Применим полученные знания на практике. Возьмем данные diamonds для работы в этой задаче. Создайте новый dataframe с именем high_price, в котором будут хранится только 10 самых дорогих бриллиантов каждого цвета. Также в итоговом datafrmae должны храниться только две переменные color и price.
```
head(high_price)
  color price
  <ord> <int>
1     D 18693
2     D 18674
3     D 18575
4     D 18542
5     D 18526
6     D 18468
```

In [0]:
%%R
high_price <- diamonds %>% 
  select(color, price) %>% 
  group_by(color) %>% 
  arrange(color, -price) %>% 
  slice(1:10)